In [1]:
from dotenv import load_dotenv
import openai
import os
from llama_index.core import (
    Settings,
    VectorStoreIndex,
    SimpleDirectoryReader, 
    StorageContext,
    load_index_from_storage,
    Document,
    PromptTemplate
)
load_dotenv()

True

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-4o-mini")
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")
Settings.embed_model = embed_model
Settings.llm = llm

In [3]:
def get_artist_documents(filename: str) -> list[Document]:
    with open(filename) as file:
        data = file.read()
    songs = data.split("===")
    artist = songs.pop(0).strip()
    
    documents = [
        Document(
            text=song,
            metadata={
                "category":"music",
                "artist": artist,
            }
        )
        for song in songs
    ]    
    return documents

In [4]:
import os
PERSIST_DIR = "lyrics_store2"

if not os.path.exists(PERSIST_DIR):
    documents = get_artist_documents("8988_Kjarkas.txt")
    index = VectorStoreIndex.from_documents(documents, show_progress=True)
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR) 
    index = load_index_from_storage(storage_context)

In [5]:
query_engine = index.as_query_engine()

In [6]:
qa_template_str = """
    You are an expert in Bolivian Folk music, your task is to guide and teach the user 
    about your field. Answer the user queries only with supported data in your context.
    Your context may contain complete lyrics or parts of them in different languages, but
    your answer will always be in Spanish. 

    Context information is below.
    ---------------------
    {context_str}
    ---------------------
    Given the context information and not prior knowledge, 
    answer the query with detailed source information, include direct quotes and use bullet lists in your 
    answers, in one of the bullets detail the tone/sentiment of the song.
    Query: {query_str}
    Answer: 
"""
qa_template = PromptTemplate(qa_template_str)

In [7]:
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_template}
)

In [8]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata

description = """
A set of lyrics for songs from the Bolivian Folk Group Los Kjarkas. 
Use plain text question as input to the tool. 
MANDATORY: Pass the response to the user as is, mantaining the format, do not try to summarize when using this tool.
"""
tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="Kjarkas_songs_lyrics",
            description=description,
            return_direct=False
        )
    )
]
agent = OpenAIAgent.from_tools(tools=tools, verbose=True)

In [ ]:
response = agent.chat("que canciones de los kjarkas hablan de traicion?")

Added user message to memory: que canciones de los kjarkas hablan de traicion?
=== Calling Function ===
Calling function: Kjarkas_songs_lyrics with args: {"input":"canciones de los kjarkas que hablan de traicion"}
Got output: Las canciones de Kjarkas que hablan de traición incluyen:

- **"Mi pecado"**
  - Esta canción trata sobre el dolor de haber creído en las palabras de alguien que resultó ser engañoso. 
  - Frases clave:
    - "Mi pecado fue quererte y haber caído en tu trampa"
    - "Yo creí en tus palabras sin saber que me engañabas"
  - **Sentimiento**: La canción expresa un profundo sentimiento de traición y desilusión, pero también una disposición a amar nuevamente a pesar del dolor.

- **"Tiempo Al Tiempo"**
  - Aunque esta canción no trata directamente sobre la traición, sí refleja el dolor y la decepción que pueden surgir de relaciones complicadas.
  - Frases clave:
    - "Me duelen tus palabras con que mastas mis ilusiones"
    - "Le daremos tiempo al tiempo, él nos dirá q

In [10]:
from IPython.display import Markdown, display
display(Markdown(response.response))

Las canciones de Los Kjarkas que hablan de traición incluyen:

- **"Mi pecado"**
  - Esta canción trata sobre el dolor de haber creído en las palabras de alguien que resultó ser engañoso. 
  - Frases clave:
    - "Mi pecado fue quererte y haber caído en tu trampa"
    - "Yo creí en tus palabras sin saber que me engañabas"
  - **Sentimiento**: La canción expresa un profundo sentimiento de traición y desilusión, pero también una disposición a amar nuevamente a pesar del dolor.

- **"Tiempo Al Tiempo"**
  - Aunque esta canción no trata directamente sobre la traición, sí refleja el dolor y la decepción que pueden surgir de relaciones complicadas.
  - Frases clave:
    - "Me duelen tus palabras con que mastas mis ilusiones"
    - "Le daremos tiempo al tiempo, él nos dirá qué sucede"
  - **Sentimiento**: La canción tiene un tono melancólico y reflexivo, donde se siente la tristeza por las ilusiones perdidas.

Ambas canciones reflejan la complejidad de las relaciones y el impacto emocional que puede tener la traición.

In [11]:
response = agent.chat("cuales hablan sobre la naturaleza?")

Added user message to memory: cuales hablan sobre la naturaleza?
=== Calling Function ===
Calling function: Kjarkas_songs_lyrics with args: {"input":"canciones de los kjarkas que hablan sobre la naturaleza"}
Got output: Los Kjarkas son conocidos por su rica conexión con la naturaleza en sus letras. Aunque las canciones "Canto a la mujer de mi pueblo" y "Canto a la madre" no se centran exclusivamente en la naturaleza, sí incluyen referencias que evocan el entorno natural y su belleza. Aquí te detallo algunos aspectos de estas canciones que hablan sobre la naturaleza:

- **Referencias a la geografía**: 
  - En "Canto a la mujer de mi pueblo", se menciona "morena piel de montañas y pueblos perdidos", lo que sugiere una conexión profunda con el paisaje montañoso de Bolivia.
  - En "Canto a la madre", se hace eco de la misma idea con "morena piel de montañas y pueblos perdidos".

- **Elementos de la naturaleza**:
  - La letra de "Canto a la mujer de mi pueblo" menciona "Luz del fogon que ll

In [12]:
display(Markdown(response.response))

Los Kjarkas son conocidos por su rica conexión con la naturaleza en sus letras. Aunque las canciones "Canto a la mujer de mi pueblo" y "Canto a la madre" no se centran exclusivamente en la naturaleza, sí incluyen referencias que evocan el entorno natural y su belleza. Aquí te detallo algunos aspectos de estas canciones que hablan sobre la naturaleza:

- **Referencias a la geografía**: 
  - En "Canto a la mujer de mi pueblo", se menciona "morena piel de montañas y pueblos perdidos", lo que sugiere una conexión profunda con el paisaje montañoso de Bolivia.
  - En "Canto a la madre", se hace eco de la misma idea con "morena piel de montañas y pueblos perdidos".

- **Elementos de la naturaleza**:
  - La letra de "Canto a la mujer de mi pueblo" menciona "Luz del fogon que lleva fuego de amor que se aviva", lo que puede interpretarse como una referencia a la naturaleza y la vida que esta proporciona.
  - En ambas canciones, se habla de "caricias de tierra", lo que refuerza la conexión con el entorno natural.

- **Sentimiento de esperanza**:
  - Ambas canciones transmiten un tono de esperanza y ternura, como se refleja en la línea "tierno canto de esperanza" en "Canto a la madre".

En resumen, aunque no son canciones que hablen exclusivamente sobre la naturaleza, las letras de Kjarkas están impregnadas de referencias a la geografía y el entorno natural, lo que refleja su amor y respeto por la tierra.